In [ ]:
%load_ext autoreload
%autoreload 2

import os

# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from backend.models import Video, UserPreferences, ExpertRating, DjangoUser, UserInformation, EmailDomain
from backend.rating_fields import VIDEO_FIELDS
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, IFrame, display
from IPython.display import YouTubeVideo
import pandas as pd
from tqdm.auto import tqdm
import gin
from backend.rating_fields import MAX_VALUE
from matplotlib import pyplot as plt
gin.enter_interactive_mode()

from ipywidgets import FloatProgress
from IPython.display import display
from functools import partial
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import logging
from django_react.settings import load_gin_config
from backend.ml_model.preference_aggregation_featureless_online import FeaturelessOnlineUpdater, compute_online_update
from backend.ml_model.preference_aggregation_featureless_online_db import OnlineRatingUpdateContext

# Using Updater class

In [ ]:
load_gin_config('../backend/backend/ml_model/config/featureless_config.gin')

In [ ]:
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

In [ ]:
# rating to change
user_set_value = 'sergei'
rating_to_change = ExpertRating.objects.filter(user__user__username=user_set_value).order_by('?')[0]
video1 = rating_to_change.video_1
video2 = rating_to_change.video_2
field_to_change = VIDEO_FIELDS[0]

In [ ]:
# database stats
qs = Video.objects.all()
qs = qs.annotate(num_scores=Count('videorating', distinct=True))
qs = qs.annotate(num_ratings1=Count('expertrating_video_1', distinct=True))
qs = qs.annotate(num_ratings2=Count('expertrating_video_2', distinct=True))
qs = qs.annotate(num_ratings=F('num_ratings1') + F('num_ratings2'))
maxes = qs.aggregate(max_scores=Max('num_scores'), max_ratings=Max('num_ratings'))
qs = qs.annotate(_metric=F('num_ratings') / maxes['max_ratings'] )#+ F('num_scores') / maxes['max_scores'])

qs = qs.order_by('-_metric')
qs[0]._metric, qs[0].num_ratings, qs[0].num_scores

In [ ]:
# data from the databse
update_context = OnlineRatingUpdateContext(rating_to_change, field_to_change)

In [ ]:
# %timeit OnlineRatingUpdateContext(rating_to_change, field_to_change)
# need to cache the model tensor
# and invalidate after some time (or when someone writes into the db)

In [ ]:
gin.bind_parameter('FeaturelessOnlineUpdater.golden_params',
                  {'maxiter': 20, 'tol': 1e-8, 'smartbracket': (-2, 2)})

In [ ]:
print("### Online updates demo")

print("Changing", update_context.feature, "on", update_context.video1, '/', update_context.video2)
print("Rating by", update_context.my_username)
print("Other users", len(update_context.usernames) - 1)
print("Ratings in the loss", len(update_context.ratings_selected))
print("Videos in the loss", len(update_context.videos_selected))

PB_MIN_SCORE, PB_MAX_SCORE = -5, 5

users_get_value = range(len(update_context.usernames) + 1)
pbars = []
pbars_v1 = []
pbars_v2 = []
for username in update_context.usernames + ['aggregated']:
    pbar = FloatProgress(min=0, max=100, description=username, bar_style='success')
    pbars.append(pbar)
    
    pbar_v1 = FloatProgress(min=PB_MIN_SCORE,
                            max=PB_MAX_SCORE, description=f'L {username}')
    pbars_v1.append(pbar_v1)
    
    pbar_v2 = FloatProgress(min=PB_MIN_SCORE,
                            max=PB_MAX_SCORE, description=f'R {username}')
    pbars_v2.append(pbar_v2)
    
for pbar in pbars:
    display(pbar)
    
for idx in range(len(pbars_v1)):
    display(pbars_v1[idx])
    display(pbars_v2[idx])

initial_rating = update_context.mb_np['cmp'][update_context.idx_set][0]
interact(compute_online_update, rating_value=widgets.FloatSlider(min=-1, max=1, step=0.05,
                                                                 value=initial_rating),
         pbars=fixed({'comparison': pbars,
                     'v1': pbars_v1, 'v2': pbars_v2}), mb_np_orig=fixed(update_context.mb_np),
         model_tensor_orig=fixed(update_context.model_tensor),
         idx_set=fixed(update_context.idx_set), users_get_value=fixed(users_get_value),
         maxiter=20, n_repeats=1, tol=fixed(1e-8),
         hotfix_update_hypers=fixed({})) # {'mu': 1, 'lambda_': 0.1}

In [ ]:
update_context.write_updates_to_db(update_context.model_tensor)